In [1]:
import os
import sys
import datetime
import time
import glob

sys.path.append('../')

import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

from src.utils.coin_prices import get_historical_price_coingecko
from src.get_block_positions import get_eth_tricrypto_positions


# Calculate


In [2]:
lp_balance = 100
cache_dir = "../data/block_positions_for_amount"
alchemy_node = f"https://eth-mainnet.alchemyapi.io/v2/{os.environ['ALCHEMY_API_KEY']}"

if not os.path.exists(cache_dir):
    os.mkdir(cache_dir)

now get underlying tokens if 100 lp tokens were removed. this will only consider addrs wo had at a minimum of 100 lp tokens in either a curve or a convex gauge.

The code can break mid way so just re-run it. it wont re-calculate positions for a block, so if it was saved in the cache_dir, it will just continue on from there till current block. If the code goes to sleep for 1800 seconds, then stop the notebook.

In [3]:
get_eth_tricrypto_positions(cache_dir=cache_dir, node_provider_https=alchemy_node, block_steps=20000, lp_amount=lp_balance)

2021-09-11:00:43:26,162-4s INFO [get_position_multicall.py get_position_multicall:22] :: Initialising Position Calculator ...
2021-09-11:00:43:27,532-4s INFO [get_position_multicall.py get_position_multicall:73] :: ... done!
2021-09-11:00:43:28,5-4s INFO [get_block_positions.py get_block_positions:114] :: Current block: 13200787
2021-09-11:00:43:28,5-4s INFO [get_block_positions.py get_block_positions:118] :: Reached max block height 13200787


In [4]:
positions_data_files = glob.glob(os.path.join(cache_dir, f"*_{lp_balance}.csv"))
positions_data = pd.DataFrame()
for filename in positions_data_files:
    csv_data = pd.read_csv(filename)
    positions_data = pd.concat([positions_data, csv_data])

positions_data

,Unnamed: 0,addr,Tether USD,Wrapped BTC,Wrapped Ether,lp_balances,block_number
0,2021-08-04 23:45:53,0xf032f533a8814741345215a0710b0364953387a5,142497.522072,3.589493,52.358654,1.000000e+20,12961148
1,2021-08-04 23:45:53,0xee28b8bf533ce188212a449e09c4bd1ed390b85e,142497.522072,3.589493,52.358654,1.000000e+20,12961148
2,2021-08-04 23:45:53,0x7217a656bd5442ad6aa51014e19846f4eaf42fe4,142497.522072,3.589493,52.358654,1.000000e+20,12961148
3,2021-08-04 23:45:53,0x34921ddccd844d0cd7b30b473889cefeae138411,142497.522072,3.589493,52.358654,1.000000e+20,12961148
4,2021-08-04 23:45:53,0xf96da4775776ea43c42795b116c7a6eccd6e71b5,142497.522072,3.589493,52.358654,1.000000e+20,12961148
...,...,...,...,...,...,...,...
27,2021-08-11 04:14:39,0xf43316162d4594d2f927a25ab75cfcac676d3e85,157889.305961,3.449626,49.786218,1.000000e+20,13001148
28,2021-08-11 04:14:39,0x3907315e7e7073015bcba729a2632fe7a02f485f,157889.305961,3.449626,49.786218,1.000000e+20,13001148
29,2021-08-11 04:14:39,0x63495284b373224c88db0354f043f7f3584b9dc6,157889.305961,3.449626,49.786218,1.000000e+20,13001148
30,2021-08-11 04:14:39,0xb18fbfe3d34fdc227eb4508cde437412b6233121,157889.305961,3.449626,49.786218,1.000000e+20,13001148


In [5]:
positions_data = positions_data.rename(columns={positions_data.columns[0]: "Timestamp"})
positions_data = positions_data.set_index(positions_data[positions_data.columns[0]])
positions_data = positions_data.loc[:, positions_data.columns != 'Timestamp']
positions_data.sort_index(inplace=True)
positions_data = positions_data[positions_data["Tether USD"] != 'Error']
positions_data['lp_balances'] = positions_data['lp_balances'].astype(float) * 1e-18
positions_data['Tether USD'] = positions_data['Tether USD'].astype(float)
positions_data['Wrapped BTC'] = positions_data['Wrapped BTC'].astype(float)
positions_data['Wrapped Ether'] = positions_data['Wrapped Ether'].astype(float)
positions_data = positions_data.rename(columns={"Wrapped Ether": "eth", "Wrapped BTC": "btc", "Tether USD": "usdt"})
positions_data


,addr,usdt,btc,eth,lp_balances,block_number
Timestamp,,,,,,
2021-07-17 02:40:00,0x7a16ff8270133f063aab6c9977183d9e72835428,116182.340403,3.698717,61.990845,100.0,12841148
2021-07-20 05:54:52,0x7a16ff8270133f063aab6c9977183d9e72835428,110670.801955,3.734523,64.144125,100.0,12861148
2021-07-20 05:54:52,0xd5431fecbb14f20f8ee68360b586d84d589f9f77,110670.801955,3.734523,64.144125,100.0,12861148
2021-07-23 08:57:01,0x1dc2dc7110f8ee61650864ad6680fe8144b7d6f9,122192.513826,3.740502,58.870186,100.0,12881148
2021-07-23 08:57:01,0xee28b8bf533ce188212a449e09c4bd1ed390b85e,122192.513826,3.740502,58.870186,100.0,12881148
...,...,...,...,...,...,...
2021-09-11 00:39:49,0xe24286adfc053f76888aa51d9a94f6c1519b4cba,157645.046685,3.517125,49.060290,100.0,13200768
2021-09-11 00:39:49,0x7a16ff8270133f063aab6c9977183d9e72835428,157645.046685,3.517125,49.060290,100.0,13200768
2021-09-11 00:39:49,0xa2694887f13a7bfee00824ba02b38ae848779136,157645.046685,3.517125,49.060290,100.0,13200768


We look at at analytics for one address.

In [6]:
addr = "0x7a16ff8270133f063aab6c9977183d9e72835428"

In [7]:
addr_positions = positions_data[positions_data.addr == addr]
addr_positions

,addr,usdt,btc,eth,lp_balances,block_number
Timestamp,,,,,,
2021-07-17 02:40:00,0x7a16ff8270133f063aab6c9977183d9e72835428,116182.340403,3.698717,61.990845,100.0,12841148
2021-07-20 05:54:52,0x7a16ff8270133f063aab6c9977183d9e72835428,110670.801955,3.734523,64.144125,100.0,12861148
2021-07-23 08:57:01,0x7a16ff8270133f063aab6c9977183d9e72835428,122192.513826,3.740502,58.870186,100.0,12881148
2021-07-26 11:50:36,0x7a16ff8270133f063aab6c9977183d9e72835428,133581.268074,3.492233,57.140308,100.0,12901148
2021-07-29 16:13:16,0x7a16ff8270133f063aab6c9977183d9e72835428,135361.997511,3.394767,58.582984,100.0,12921148
2021-08-01 20:12:44,0x7a16ff8270133f063aab6c9977183d9e72835428,141717.510245,3.448818,55.123014,100.0,12941148
2021-08-04 23:45:53,0x7a16ff8270133f063aab6c9977183d9e72835428,142497.522072,3.589493,52.358654,100.0,12961148
2021-08-08 02:18:24,0x7a16ff8270133f063aab6c9977183d9e72835428,156030.729903,3.506792,49.445598,100.0,12981148
2021-08-11 04:14:39,0x7a16ff8270133f063aab6c9977183d9e72835428,157889.305961,3.449626,49.786218,100.0,13001148


We need a price column

In [8]:
wbtc = "0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599"
weth = "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"

In [9]:
addr_positions['eth_usd'] = 0
addr_positions['btc_usd'] = 0

for index, row in addr_positions.iterrows():
    timestamp = datetime.datetime.strptime(index, '%Y-%m-%d %H:%M:%S')
    eth_price = get_historical_price_coingecko(weth, timestamp).quote
    btc_price = get_historical_price_coingecko(wbtc, timestamp).quote
    addr_positions.loc[index, 'eth_usd'] = eth_price * row.eth
    addr_positions.loc[index, 'btc_usd'] = btc_price * row.btc

addr_positions


/Users/swadhinnanda/Projects/git/convex-community/tricrypto-analysis/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/swadhinnanda/Projects/git/convex-community/tricrypto-analysis/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/swadhinnanda/Projects/git/convex-community/tricrypto-analysis/venv/lib/python3.7/site-packages/pandas/core/ind

HTTPError: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/coins/ethereum/contract/0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2/market_chart/range?vs_currency=usd&from=1631313589&to=1631318389

impermanent loss is dollar value of each of the coins at the start vs today (at today's prices) 

In [ ]:
initial_btc_value = addr_positions.loc[addr_positions.index[0], 'btc'] * btc_price
initial_eth_value = addr_positions.loc[addr_positions.index[0], 'eth'] * eth_price
initial_usd_value = addr_positions.loc[addr_positions.index[0], 'usdt']

print(initial_usd_value, initial_btc_value, initial_eth_value)

HTTPError: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/coins/ethereum/contract/0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2/market_chart/range?vs_currency=usd&from=1631313589&to=1631318929

116182.340403 171995.9980806562 208148.15678952393


In [ ]:
final_btc_value = addr_positions.loc[addr_positions.index[-1], 'btc'] * btc_price
final_eth_value = addr_positions.loc[addr_positions.index[-1], 'eth'] * eth_price
final_usd_value = addr_positions.loc[addr_positions.index[-1], 'usdt']

print(final_usd_value, final_btc_value, final_eth_value)

165041.110201 165489.97580863975 165378.14447965368


In [ ]:
print(f'impermanent loss usd: {(final_usd_value - initial_usd_value)/initial_usd_value * 100}%')
print(f'impermanent loss btc: {(final_btc_value - initial_btc_value)/initial_btc_value * 100}%')
print(f'impermanent loss eth: {(final_eth_value - initial_eth_value)/initial_eth_value * 100}%')

impermanent loss usd: 42.05352519885923%
impermanent loss btc: -4.86460859048457%
impermanent loss eth: -24.42948988241428%
